### Lab 2. Linear programming

1. Implement an option to enter data from a JSON file.
2. If necessary, add balancing variables to make a transition from the general formulation to the canonical form of the problem of linear
programming
3. Use the artificial basis method, if necessary, to select the primary reference plan.
4. Implement the simplex method to solve the problem.
5. Consider that the problem may have no solutions at all,
and have an infinite number of solutions.
6. Implement sequential output of simplex tables at all steps of the algorithm.

#### 1

Implement an option to enter data from a JSON file.

In [13]:
from pathlib import Path
import numpy as np
import orjson as json


class problem:
    f_coefs = "f"
    f_goal = "goal"
    goal_min = "min"
    goal_max = "max"
    f_constraints = "constraints"
    constraint_coefs = "coefs"
    constraint_type = "type"
    type_lte = "lte"
    type_gte = "gte"
    type_eq = "eq"
    constraint_b = "b"

    def __init__(self):
        self.problem_description: dict = None

    def parse_from_json(self, s: str | Path) -> bool:
        if isinstance(s, Path):
            with open(s, "rb") as f:
                s = f.read()
        try:
            problem_description: dict = json.loads(s)
        except:
            problem_description = None
            print("Error on problem parsing: s = ", s)
        self.problem_description = problem_description
        return problem is not None

    def is_problem(self) -> bool:
        """check that all the necessary fields exits
        and have the values have the appropriate types"""
        import numbers

        pd = self.problem_description
        if pd is None:
            print("problem description does not exits")
            return False
        fields_list = [problem.f_coefs, problem.f_goal, problem.f_constraints]
        for field in fields_list:
            if field not in pd:
                print(f"field '{field}' does not exits")
                return False
        if (
            not isinstance(pd[problem.f_coefs], list)
            or not isinstance(pd[problem.f_goal], str)
            or not isinstance(pd[problem.f_constraints], list)
        ):
            return False
        for coef in pd[problem.f_coefs]:
            if not isinstance(coef, numbers.Number):
                print(f"coef '{coef}' is not numeric type")
                return False

        constraints_fields_list = [
            problem.constraint_b,
            problem.constraint_type,
            problem.constraint_coefs,
        ]
        for constraint in pd[problem.f_constraints]:
            for field in constraints_fields_list:
                if field not in constraint:
                    print(
                        f"field '{field}' in constraint '{constraint}' does not exits"
                    )
                    return False
            if (
                not isinstance(constraint[problem.constraint_b], numbers.Number)
                or not isinstance(constraint[problem.constraint_type], str)
                or not isinstance(constraint[problem.constraint_coefs], list)
            ):
                print(f"constraint '{constraint}' has wrong format")
                return False
            for coef in constraint[problem.constraint_coefs]:
                if not isinstance(coef, numbers.Number):
                    print(
                        f"coef '{coef}' in constraint '{constraint}' is not numeric type"
                    )
                    return False
        return True

In [15]:
def test_input():
    json_text = R"""{ "f": [1, 2, 3],
"goal": "max",
"constraints": [{"coefs": [1, 0, 0],
"type": "lte",
"b": 1},
{"coefs": [1, 1, 0],
"type": "gte",
"b": 2},
{"coefs": [1, 1, 1],
"type": "eq",
"b": 3}]}"""
    a = problem()
    assert a.parse_from_json(json_text)
    # print(a.problem_description)
    assert a.is_problem()
test_input()

#### 2
If necessary, add balancing variables to make a transition from the general formulation to the canonical form of the problem of linear
programming


In [17]:
def is_canonical_form(self) -> bool:
    if not self.is_problem():
        return False
    if self.problem_description[problem.f_goal] == problem.goal_max:
        return False
    for constraint in self.problem_description[problem.f_constraints]:
        if constraint[problem.constraint_type] != problem.type_eq:
            return False
    return True


problem.is_canonical_form = is_canonical_form


def to_canonical_form(self) -> "problem":
    """f->min + constraints: only eq"""
    if not self.is_problem():
        return problem()
    p = problem()
    p.problem_description = self.problem_description.copy()
    pd = p.problem_description

    if self.is_canonical_form():
        return p

    if pd[problem.f_goal] == problem.goal_max:
        pd[problem.f_coefs] = [-coef for coef in pd[problem.f_coefs]]
        pd[problem.f_goal] = problem.goal_min
    for constraint in pd[problem.f_constraints]:
        if constraint[problem.constraint_type] == problem.type_eq:
            continue
        # add balancing vars
        elif constraint[problem.constraint_type] == problem.type_gte:
            constraint[problem.constraint_coefs].append(-1)
        elif constraint[problem.constraint_type] == problem.type_lte:
            constraint[problem.constraint_coefs].append(1)
        constraint[problem.constraint_type] = problem.type_eq
    return p


problem.to_canonical_form = to_canonical_form

In [20]:
def test_canonical_form():
    json_text = R"""{ "f": [1, 2, 3],
"goal": "max",
"constraints": [{"coefs": [1, 0, 0],
"type": "lte",
"b": 1},
{"coefs": [1, 1, 0],
"type": "gte",
"b": 2},
{"coefs": [1, 1, 1],
"type": "eq",
"b": 3}]}"""
    a = problem()
    assert a.parse_from_json(json_text)
    # print(a.problem_description)
    assert a.is_problem()
    assert not a.is_canonical_form()
    b = a.to_canonical_form()
    print(b.problem_description)
    assert b.is_canonical_form()


test_canonical_form()

{'f': [-1, -2, -3], 'goal': 'min', 'constraints': [{'coefs': [1, 0, 0, 1], 'type': 'eq', 'b': 1}, {'coefs': [1, 1, 0, -1], 'type': 'eq', 'b': 2}, {'coefs': [1, 1, 1], 'type': 'eq', 'b': 3}]}


#### 3
Use the artificial basis method, if necessary, to select the primary reference plan.


#### 4
Implement the simplex method to solve the problem.


#### 5
Consider that the problem may have no solutions at all,
and have an infinite number of solutions.


#### 6
Implement sequential output of simplex tables at all steps of the algorithm.